# FLUX LoRA Training in Google Colab

이 노트북은 Google Colab에서 FLUX LoRA 훈련을 위한 환경설정 및 실행 가이드입니다.

In [ ]:
# GPU 정보 확인
!nvidia-smi
!nvcc --version

In [ ]:
# 필요한 패키지 설치 (CUDA 호환성 확보)
# PyTorch와 torchvision CUDA 버전 통일

# 기존 설치된 패키지 제거 (충돌 방지)
!pip uninstall torch torchvision torchaudio -y

# 호환되는 PyTorch 버전 설치 (CUDA 11.8)
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

# 다른 필수 패키지들
!pip install diffusers transformers peft accelerate datasets
!pip install Pillow pandas numpy tqdm wandb safetensors
!pip install sentencepiece protobuf python-dotenv

# 설치 확인
import torch
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ CUDA version: {torch.version.cuda}")
    print(f"✅ GPU: {torch.cuda.get_device_name()}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

In [ ]:
# 환경변수 및 GitHub 토큰 설정
import os
from google.colab import userdata

# 방법 1: Colab Secrets 사용 (권장)
# 🔑 키 아이콘 클릭 → Add new secret:
# HUGGINGFACE_TOKEN: hf_your_token_here
# GITHUB_TOKEN: ghp_your_token_here (Private 저장소용)

try:
    HF_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    
    os.environ['HUGGINGFACE_TOKEN'] = HF_TOKEN
    os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
    
    print("✅ Tokens loaded from Colab secrets")
except:
    # 방법 2: 직접 입력 (임시용)
    HF_TOKEN = "hf_your_token_here"
    GITHUB_TOKEN = "ghp_your_token_here"
    
    os.environ['HUGGINGFACE_TOKEN'] = HF_TOKEN
    os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
    
    print("⚠️ Tokens set manually")

# 기타 환경변수 설정
os.environ['FLUX_MODEL_NAME'] = 'black-forest-labs/FLUX.1-schnell'
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
print("✅ Environment variables configured")

In [ ]:
# Private GitHub 저장소 클로닝
import os

# GitHub 토큰 확인
github_token = os.environ.get('GITHUB_TOKEN')

if github_token and github_token != "ghp_your_token_here":
    # 토큰을 사용한 Private 저장소 클로닝
    !git clone https://{github_token}@github.com/comking2/flux-lora-training.git
    print("✅ Private repository cloned successfully")
else:
    # 토큰이 없으면 Public 저장소 시도 또는 수동 업로드 안내
    try:
        !git clone https://github.com/comking2/flux-lora-training.git
        print("✅ Public repository cloned")
    except:
        print("❌ Repository access failed")
        print("💡 해결방법:")
        print("1. GitHub Token을 Colab Secrets에 추가")
        print("2. 또는 저장소를 Public으로 변경")
        print("3. 또는 파일을 직접 업로드")

# 클로닝된 디렉토리로 이동
try:
    %cd flux-lora-training
    print("📁 Moved to project directory")
    
    # 파일 목록 확인
    !ls -la
except:
    print("⚠️ Directory not found - please check cloning status")

In [ ]:
# 대안: 개별 파일 다운로드 (클로닝 실패 시)
# GitHub Token이 없거나 Private 저장소 접근이 안 될 때 사용

print("🔄 Alternative: Downloading individual files...")

# 주요 파일들을 개별적으로 다운로드
files_to_download = [
    'train_lora.py',
    'dataset_processor.py', 
    'inference.py',
    'config.json',
    'requirements.txt',
    '.env.example'
]

import requests
import os

def download_file(filename, token=None):
    if token:
        url = f"https://api.github.com/repos/comking2/flux-lora-training/contents/{filename}"
        headers = {'Authorization': f'token {token}'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            import base64
            content = base64.b64decode(response.json()['content']).decode('utf-8')
            with open(filename, 'w') as f:
                f.write(content)
            return True
    return False

# GitHub API를 통한 다운로드 시도
github_token = os.environ.get('GITHUB_TOKEN')
downloaded = []

for filename in files_to_download:
    if download_file(filename, github_token):
        downloaded.append(filename)
        print(f"✅ Downloaded: {filename}")
    else:
        print(f"❌ Failed: {filename}")

if downloaded:
    print(f"\\n✅ Successfully downloaded {len(downloaded)} files")
    !ls -la
else:
    print("\\n💡 Please upload files manually or provide GitHub token")

In [ ]:
# 데이터셋 업로드 또는 다운로드
# 방법 1: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 방법 2: 직접 업로드
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# GPU 메모리 최적화 설정
import torch

# GPU 메모리 확인
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    
    # 메모리 캐시 정리
    torch.cuda.empty_cache()

In [ ]:
# FLUX LoRA 훈련 실행
# T4 (16GB) 환경 최적화

!python train_lora.py \
    --data_dir "/content/drive/MyDrive/training_data" \
    --output_dir "./flux_lora_output" \
    --epochs 10 \
    --batch_size 1 \
    --lora_rank 8 \
    --learning_rate 5e-5